# 신경망 학습의 순서

**전제**
- 신경망 학습에는 적응 가능한 가중치와 편향이 있다.
- 가중치와 편향을 푼련 데이터에 적응하도록 조정하는 과정을 **학습**이라고 한다.
- 학습은 다음 4단계로 진행된다.

**1단계 - 미니배치**
- 훈련 데이터 중 일부를 무작위로 가져온다.
- 이렇게 선별한 데이터를 미니배치라고 한다.
- 미니배치의 손실함수 값을 줄이는 것이 목표이다.

**2단계 - 기울기 산출**
- 미니배치의 손실함수 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구한다.
- 기울기는 손실함수의 값을 가장 작게 하는 방향을 제시한다.

**3단계 - 매개변수 갱신**
- 가중치 매개변수를 기울기 방향으로 아주 조금 갱신한다.

**4단계 - 반복**
- 1~3단계를 반복한다.

-----

# IMPORT

In [1]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from Common.layers import *
from Common.gradient import numerical_gradient
from collections import OrderedDict

In [2]:
from Dataset.MNIST.mnist import load_mnist

# 구현하기

In [3]:
class Affine:
    def __init__(self, W, b):
        self.W =W
        self.b = b
        
        self.x = None
        self.original_x_shape = None
        # 重み・バイアスパラメータの微分
        self.dW = None
        self.db = None

    def forward(self, x):
        # テンソル対応
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x

        out = np.dot(self.x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        dx = dx.reshape(*self.original_x_shape)  # 入力データの形状に戻す（テンソル対応）
        return dx

## 2층 신경망

In [4]:
class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 가중치의 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
        
    # x: 입력 데이터, t: 정답 라벨
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x: 입력 데이터, t: 정답 라벨
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        # 순전파
        self.loss(x, t)

        # 역전파
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

## 기울기 검증

In [5]:
# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 각 가중치의 차이의 절댓값을 구한 후 절댓값들의 평균을 구한다.
for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(key + ":" + str(diff))

W1:4.0691212296991757e-10
b1:2.4601384499521597e-09
W2:5.7419309359415325e-09
b2:1.402419795501486e-07


## 학습 구현

In [6]:
# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 오차역전파법으로 기울기를 구한다.
    grad = network.gradient(x_batch, t_batch)
    
    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        print("----------")
        print(f"에포크: {i}")
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(f"학  습  정확도: {train_acc:.4f}\n테스트 정확도:  {test_acc:.4f}")
        print("----------")

----------
에포크: 0
학  습  정확도: 0.1245
테스트 정확도:  0.1194
----------
----------
에포크: 600
학  습  정확도: 0.9004
테스트 정확도:  0.9018
----------
----------
에포크: 1200
학  습  정확도: 0.9238
테스트 정확도:  0.9245
----------
----------
에포크: 1800
학  습  정확도: 0.9351
테스트 정확도:  0.9345
----------
----------
에포크: 2400
학  습  정확도: 0.9437
테스트 정확도:  0.9396
----------
----------
에포크: 3000
학  습  정확도: 0.9505
테스트 정확도:  0.9472
----------
----------
에포크: 3600
학  습  정확도: 0.9574
테스트 정확도:  0.9539
----------
----------
에포크: 4200
학  습  정확도: 0.9618
테스트 정확도:  0.9576
----------
----------
에포크: 4800
학  습  정확도: 0.9650
테스트 정확도:  0.9604
----------
----------
에포크: 5400
학  습  정확도: 0.9676
테스트 정확도:  0.9622
----------
----------
에포크: 6000
학  습  정확도: 0.9695
테스트 정확도:  0.9638
----------
----------
에포크: 6600
학  습  정확도: 0.9717
테스트 정확도:  0.9661
----------
----------
에포크: 7200
학  습  정확도: 0.9740
테스트 정확도:  0.9674
----------
----------
에포크: 7800
학  습  정확도: 0.9751
테스트 정확도:  0.9682
----------
----------
에포크: 8400
학  습  정확도: 0.9771
테스트 정확도:  0.9702
----------